In [9]:
import numpy as np
from numba import jit
from copy import copy
import progressbar
import math

In [10]:
CALIBRON_PIECES = [(21, 14),
                   (17, 14),
                   (21, 18),
                   (32, 10),
                   (21, 14),
                   (10, 7),
                   (14, 4),
                   (21, 18),
                   (28, 6),
                   (28, 14),
                   (32, 11),
                   (28, 7)]

In [11]:
WIDTH, HEIGHT = 56, 56
LENGTH = len(CALIBRON_PIECES) - 1
FACT = {x: math.factorial(x) * 2 ** x for x in range(0, LENGTH + 1)}

In [32]:
def newBoard(width, height):
        return np.array([[False for i in range(height)] for j in range(width)])       

    
# @jit(nopython=True)
def piece_valid(largest_piece, piece, smallest_side):
    return ((piece[0] <= largest_piece[0]) and
            (piece[1] <= largest_piece[1]) and 
            (HEIGHT - position[0] - piece[0] >= smallest_side or HEIGHT - position[0] - piece[0] == 0) and
            (WIDTH - position[1] - piece[1] >= smallest_side or WIDTH - position[1] - piece[1] == 0))


@jit(nopython=True)
def place_piece(board, position, piece):
    board[position[1]:position[1] + piece[1],
        position[0]:position[0] + piece[0]] = True

    
@jit(nopython=True)
def remove_piece(board, position, piece):
    board[position[1]:position[1] + piece[1],
        position[0]:position[0] + piece[0]] = False

    
@jit(nopython=True)
def update_position(board, position, width, height):
    # first check current row
    i = position[1]
    for j in range(position[0], width):
        if not board[i][j]:
            return (j, i)
    # then check remaining rows
    for i in range(position[1] + 1, height):
        for j in range(width):
            if not board[i][j]:
                return (j, i)


@jit(nopython=True)
def update_largest_piece(width, height, board, position):
    piece_height = height - position[1]
    piece_width = 0
    for j in range(position[0], width):
        if not board[position[1]][j]:
            piece_width += 1
        else:
            break
    return (piece_width, piece_height)


def display(board) -> None:
    for row in board:
        for cell in row:
            if cell:
                print(" x", end="")
            else:
                print(" _",end="")
        print("")
    print()


def reverse(piece):
    yield(piece)
    yield((piece[1], piece[0]))


def update_progress(n):
    global BAR
    global COUNT
    COUNT += FACT[n]
    if COUNT % 100000 == False:
        BAR.update(COUNT)
        
def is_end(i, length):
    return i == length

In [33]:
def recurse(i, pieces, board, position):
    largest_piece = update_largest_piece(WIDTH, HEIGHT, board, position)
    smallest_side = min((min(piece) for piece in pieces[i:LENGTH + 1]))
    j = LENGTH
    while i <= j:
        for piece in reverse(pieces[i]):
            if piece_valid(largest_piece, piece, smallest_side):
                place_piece(board, position, piece)
                if is_end(i, LENGTH):
                    return pieces
                solution = recurse(i + 1, copy(pieces), board, update_position(board, position, WIDTH, HEIGHT))
                if solution:
                    return solution
                else:
                    remove_piece(board, position, piece)
            else:   
                update_progress(LENGTH - i)
        if i != j:
            pieces[i], pieces[j] = pieces[j], pieces[i]
        j -= 1
    return None

In [34]:
BAR = progressbar.ProgressBar()
COUNT = 0
solution = recurse(0, CALIBRON_PIECES, newBoard(WIDTH, HEIGHT), (0,0))
print(solution)

- |               #                         | 60180000000 Elapsed Time: 0:00:06

[(21, 14), (21, 14), (28, 14), (32, 10), (28, 6), (14, 4), (32, 11), (21, 18), (21, 18), (17, 14), (10, 7), (28, 7)]
